# Automated ComfyUI on Brev

## Source

This notebook is modified from the Brev notebook for ComfyUI

## About

This notebook will automate the install of ComfyUI on Brev. It will use the Burrito Cat AI CivitAI downloader for any CivitAI models you want to use. This will help reduce your run time and decrease costs as you don't have to set it all up.

Make sure you export your CivitAI API key to the environment variables using a `.env` file.

List your custom nodes in the OPTIONS section below. Include Manager if you want it.

In order to run this...

In [ ]:
%pip install python-dotenv
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121
%pip install insightface onnxruntime onnxruntime-gpu huggingface_hub[cli]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Environment Setup

from pathlib import Path
import os
from dotenv import load_dotenv


load_dotenv()


OPTIONS = {}

WORKSPACE = 'ComfyUI'

#@markdown **Custom Nodes Repositories**
NODES_REPOS = [
    'https://github.com/ltdrdata/ComfyUI-Manager.git',
    'https://github.com/ltdrdata/ComfyUI-Impact-Pack.git',
    'https://github.com/ltdrdata/ComfyUI-Impact-Subpack.git',
    'https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git'
]

CIVITAI_API_KEY=os.environ.get('CIVITAI_API_KEY')

CIVITAI_MODELS = [
    'urn:air:flux1:checkpoint:civitai:618692@691639',
    'urn:air:flux1:vae:civitai:636193@711305'
]

HUGGINGFACE_MODELS = []

OPTIONS['NODES_REPOS'] = NODES_REPOS
OPTIONS['CIVITAI_MODELS'] = CIVITAI_MODELS
OPTIONS['HUGGINGFACE_MODELS'] = HUGGINGFACE_MODELS

In [3]:
# Helper to download repos

import os
import re
import subprocess
import time
from typing import List, Optional

def clone_repo(repo_url: str, target_dir: str) -> None:
    """Clone a Git repository to the target directory."""
    # Extract repo name from URL (works with both HTTPS and SSH URLs)
    repo_name = None
    
    # Try to match patterns for extracting repository name
    https_pattern = r'.*/.*/(.*)\.git$'
    simple_pattern = r'.*/(.*)$'
    
    match = re.match(https_pattern, repo_url)
    if match:
        repo_name = match.group(1)
    else:
        match = re.match(simple_pattern, repo_url)
        if match:
            repo_name = match.group(1)
    
    # If we can't extract a name, use a timestamp
    if not repo_name:
        repo_name = f"custom_repo_{int(time.time())}"
    
    dest_path = os.path.join(target_dir, repo_name)
    print(f"Cloning repository: {repo_url} to {dest_path}")
    
    # Clone the repository using subprocess
    try:
        result = subprocess.run(
            ["git", "clone", repo_url, dest_path],
            check=True,
            capture_output=True,
            text=True
        )
        print(f"Successfully cloned {repo_url}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to clone {repo_url}")
        print(f"Error: {e.stderr}")

In [4]:
# Clone ComfyUI

target_dir = ""
repo = "https://github.com/comfyanonymous/ComfyUI"

clone_repo(repo, target_dir)

Cloning repository: https://github.com/comfyanonymous/ComfyUI to ComfyUI
Successfully cloned https://github.com/comfyanonymous/ComfyUI


In [5]:
# Install Requirements
import os

%pip install -r ComfyUI/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 MB 1.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 MB 1.2 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 862.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 955.9 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 807.3 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35

Now move on to the custom nodes for this install

In [6]:

# Get the target directory from environment variable or use a default
target_dir = "ComfyUI/custom_nodes"

# Check if repositories are specified in OPTIONS
if 'NODES_REPOS' in OPTIONS and OPTIONS['NODES_REPOS']:
    repos = OPTIONS['NODES_REPOS']  # This is already an array of GitHub repos
    print(f"Cloning {len(repos)} repositories from OPTIONS['NODES_REPOS']")
    
    for repo in repos:
        if repo:  # Make sure the repo URL is not empty
            clone_repo(repo, target_dir)
else:
    print("No custom nodes specified in OPTIONS['NODES_REPOS']")

Cloning 4 repositories from OPTIONS['NODES_REPOS']
Cloning repository: https://github.com/ltdrdata/ComfyUI-Manager.git to ComfyUI/custom_nodes/ComfyUI-Manager
Successfully cloned https://github.com/ltdrdata/ComfyUI-Manager.git
Cloning repository: https://github.com/ltdrdata/ComfyUI-Impact-Pack.git to ComfyUI/custom_nodes/ComfyUI-Impact-Pack
Successfully cloned https://github.com/ltdrdata/ComfyUI-Impact-Pack.git
Cloning repository: https://github.com/ltdrdata/ComfyUI-Impact-Subpack.git to ComfyUI/custom_nodes/ComfyUI-Impact-Subpack
Successfully cloned https://github.com/ltdrdata/ComfyUI-Impact-Subpack.git
Cloning repository: https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git to ComfyUI/custom_nodes/ComfyUI_Comfyroll_CustomNodes
Successfully cloned https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git


In [7]:
import os
import subprocess
import glob

# Get all directories within custom_nodes/
custom_node_dirs = glob.glob('custom_nodes/*/')

for dir_path in custom_node_dirs:
    req_file = os.path.join(dir_path, 'requirements.txt')
    
    # Check if requirements.txt exists in the directory
    if os.path.isfile(req_file):
        print(f"Installing requirements in {dir_path}")
        # Run pip install for the requirements file
        subprocess.check_call(['pip', 'install', '-r', req_file])

In [12]:
# Install bcat-civitai 

import subprocess
import os

if 'CIVITAI_MODELS' in OPTIONS and OPTIONS['CIVITAI_MODELS']:
    print("Installing bcat-civitai")

    target_dir = ""
    repo = "https://github.com/burritocatai/bcat-civitai.git"

    clone_repo(repo, target_dir)

    os.chdir("bcat-civitai")

    # Build the Rust project
    build_result = subprocess.run(["cargo", "build", "--release"], 
                                capture_output=True, text=True)

    if build_result.returncode == 0:
        print("Build successful!")
        
        for urn in OPTIONS['CIVITAI_MODELS']:
            print(f"Downloading model {urn}")
            run_result = subprocess.run(["./target/release/bcat-civitai", "--urn", urn, "--token", CIVITAI_API_KEY, "--comfyui"], 
                           capture_output=True, text=True)
            if run_result.returncode == 0:
                print(f"Model {urn} downloaded successfully!")
            else:
                print(f"Model {urn} download failed: {run_result.stderr}")
        
    else:
        print(f"Build failed: {build_result.stderr}")

    os.chdir("..")

In [13]:
import os

os.chdir("ComfyUI")

In [14]:
import subprocess
import threading
import time
import socket

def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()

In [15]:
!python main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies. (GitPython)
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-03-05 20:47:23.640
** Platform: Linux
** Python version: 3.12.9 (main, Feb 25 2025, 21:57:38) [GCC 12.2.0]
** Python executable: /usr/local/bin/python
** ComfyUI Path: /workspace/ComfyUI
** ComfyUI Base Folder Path: /workspace/ComfyUI
** User directory: /workspace/ComfyUI/user
** ComfyUI-Manager config path: /workspace/ComfyUI/user/default/ComfyUI-Manager/config.ini
** Log path: /workspace/ComfyUI/user/comfyui.log
[ComfyUI-Manager] 'numpy' dependency were fixed
[ComfyUI-Manager] 'comfyui-frontend-package' dependency were fixed

Prestartup times for custom nodes:
  28.4 seconds: /workspace/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Traceback (most recent call last):
  File "/workspace/ComfyUI/main.py", line 136, in <module>
    import execution
  File "/workspace/Comfy